# Интеграция алгоритмов поиска в систему DEGANN

## 📌 Контекст и проблема
**Последние изменения в системе:**
- Полностью переработана система конфигураций нейросетевых топологий
- Добавлен механизм метаданных для настройки параметров
- Введена новая структура хранения параметров обучения

**Возникшие сложности:**
- Существующие алгоритмы поиска (grid search и др.) перестали работать
- Невозможно быстро добавлять новые типы архитектур
- Код алгоритмов "завязан" на конкретные реализации топологий

**Почему это критично:**
- Система теряет гибкость — ключевое требование для DEGANN
- Замедляется добавление экспериментальных архитектур (GAN, Transformer)

In [ ]:
import sys
from pathlib import Path

# Путь к DEGANN через подмодуль
degann_path = Path("./deps/degann").resolve()
sys.path.append(str(degann_path))

# Проверка доступности DEGANN
try:
    from degann.networks import IModel
    print("DEGANN успешно импортирован!")
except ImportError as e:
    print(f"Ошибка: {e}\nПроверьте путь: {degann_path}")


---

## 🧪 Тестовый пример с Optuna
**Цель:** Проверить базовую интеграцию Optuna до работы с DEGANN

In [ ]:
import optuna
import math

def objective(trial):
    """Тестовая функция для проверки работы Optuna."""
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    return math.sin(x) + math.cos(y)

# Запуск оптимизации с разными алгоритмами
samplers = {
    "CMA-ES": optuna.samplers.CmaEsSampler(),
    "TPE": optuna.samplers.TPESampler(),
    "Random": optuna.samplers.RandomSampler()
}

for name, sampler in samplers.items():
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=50)
    print(f"{name}: Best value = {study.best_value:.3f}")


---

## 🚀 Переход к реальной задаче
**Цель:** Применить Optuna для поиска оптимальной GAN-архитектуры в DEGANN.

**План эксперимента:**
1. Подготовить тестовые данные (функция `sin(10x)`)
2. Определить изменяемые параметры:
   - Размеры слоев генератора/дискриминатора
   - Функции активации
   - Learning rate
3. Реализовать objective-функцию для Optuna
4. Запустить поиск с сохранением лучшей модели

**Ключевое отличие от теста:**
- Оптимизируем не детерминированную функцию, а стохастический процесс обучения
- Каждое "вычисление" — это полноценное обучение нейросети


In [ ]:
# Основные импорты
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Компоненты DEGANN
from degann.networks import IModel
from degann.networks.topology.densenet.topology_config import DenseNetParams
from degann.networks.topology.densenet.compile_config import DenseNetCompileParams
from degann.networks.topology.gan.topology_config import GANTopologyParams
from degann.networks.topology.gan.compile_config import GANCompileParams

# Генерация данных
def generate_data(n_samples=2048):
    """Создает данные для обучения GAN (аппроксимация sin(10x))."""
    X = np.linspace(0, 1, int(n_samples / 0.8)).reshape(-1, 1)
    y = np.sin(10 * X)
    return train_test_split(X, y, test_size=0.2)

X_train, X_test, y_train, y_test = generate_data()

In [ ]:
# Глобальные переменные для хранения лучшей модели
best_gan = None
best_score = float('inf')

def create_gan_config(trial):
    """Генерирует конфиг GAN на основе параметров из Optuna."""
    # Параметры генератора
    gen_layers = trial.suggest_int('gen_layers', 1, 5)
    gen_neurons = trial.suggest_categorical('gen_neurons', [16, 32, 64])
    
    # Параметры дискриминатора
    disc_layers = trial.suggest_int('disc_layers', 1, 5)
    disc_neurons = trial.suggest_categorical('disc_neurons', [16, 32, 64])
    
    # Обучение
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    
    return {
        'gen_layers': gen_layers,
        'gen_neurons': gen_neurons,
        'disc_layers': disc_layers,
        'disc_neurons': disc_neurons,
        'learning_rate': learning_rate
    }

In [ ]:
def gan_objective(trial):
    """Функция для оптимизации средствами Optuna."""
    global best_gan, best_score
    
    # 1. Получаем параметры
    params = create_gan_config(trial)
    
    # 2. Собираем GAN
    gan = build_gan(params)
    
    # 3. Обучение (упрощенное)
    history = gan.train(
        X_train, y_train,
        epochs=100,
        mini_batch_size=64,
        verbose=0
    )
    
    # 4. Оценка
    val_loss = gan.evaluate(X_test, y_test)[1]
    
    # 5. Сохранение лучшей модели
    if val_loss < best_score:
        best_score = val_loss
        best_gan = gan
        
    return val_loss

def build_gan(params: dict) -> IModel:
    """Создает и компилирует GAN модель на основе параметров.
    
    Args:
        params: Словарь с параметрами из Optuna:
            - gen_layers: int - число слоев генератора
            - gen_neurons: int - нейроны в слоях генератора
            - disc_layers: int - число слоев дискриминатора 
            - disc_neurons: int - нейроны в слоях дискриминатора
            - learning_rate: float - скорость обучения
            
    Returns:
        Скомпилированная GAN модель (IModel)
    """
    # 1. Конфиг генератора
    gen_config = DenseNetParams(
        input_size=1,
        block_size=[params['gen_neurons']] * params['gen_layers'],
        output_size=1,
        activation_func="leaky_relu"  # Можно сделать tunable
    )
    
    # 2. Конфиг дискриминатора
    disc_config = DenseNetParams(
        input_size=2,
        block_size=[params['disc_neurons']] * params['disc_layers'],
        output_size=1,
        activation_func="leaky_relu"
    )
    
    # 3. Сборка GAN
    gan_params = GANTopologyParams(
        generator_params=gen_config,
        discriminator_params=disc_config
    )
    
    # 4. Компиляция
    compile_config = GANCompileParams(
        generator_params=DenseNetCompileParams(
            rate=params['learning_rate'],
            optimizer="Adam",
            loss_func="BinaryCrossentropy"
        ),
        discriminator_params=DenseNetCompileParams(
            rate=params['learning_rate'], 
            optimizer="Adam",
            loss_func="BinaryCrossentropy"
        )
    )
    
    # 5. Создание модели
    gan = IModel(gan_params)
    gan.compile(compile_config)
    
    return gan

In [ ]:
# Закоментируйте это если хотите более подробный вывод истории подбора параметров
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Создание вспомогательных функций для вывода
def print_summary(study):
    """Красивый вывод результатов."""
    print("\n" + "="*50)
    print("🎯 Best trial:")
    for k, v in study.best_trial.params.items():
        print(f"- {k:15}: {v}")
    print(f"\n🏆 Best value: {study.best_value:.4f}")
    print("="*50)

def smart_callback(study, trial):
    """Умный вывод: только каждые 5 trials и улучшения."""
    if trial.number % 5 == 0 or study.best_trial.number == trial.number:
        msg = f"Trial {trial.number}: val_loss={trial.value:.4f}"
        if study.best_trial.number == trial.number:
            msg += " 🏆 NEW BEST"
        print(msg)


In [ ]:

# Настройка исследования
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler()
)

# Лимит: 50 trials или 1 час
study.optimize(
    gan_objective, 
    n_trials=50, 
    timeout=3600,
    callbacks=[smart_callback],
    show_progress_bar=True
)

In [ ]:
print_summary(study)

if best_gan:
    print("Модель сохранена в переменной `best_gan`")

## 🔎 Выводы из эксперимента с GAN

### Что получилось:
✅ **Интеграция Optuna работает**:  
   - Алгоритм успешно подбирает архитектуру (слои, нейроны, LR)  
   - Лучшая модель сохраняется автоматически  

✅ **Доказана концепция**:  
   - DEGANN + Optuna решают задачу оптимизации **параметров сети**  
   - **Шаблон кода** (objective + обучение) универсален, но требует:  
     - Реализации `build_model()` под конкретную архитектуру  
     - Ручного описания параметров (`suggest_int`, `suggest_float`) 

### Проблемы текущей реализации:
⚠️ **Жёсткая привязка к GAN**:  
   - Параметры прописаны вручную (`gen_layers`, `disc_neurons`)  
   - Нет унификации для других архитектур  

⚠️ **Ограниченный поиск**:  
   - Нельзя динамически менять:  
     - Число слоёв  
     - Типы слоёв (например, LSTM + Dense)  

### Требуется улучшение:
1. **Абстрактный слой** для описания параметров (→ `ConfigSampler`)  
2. **Поддержка сложных правил** (например, "от 2 до 5 слоёв с шагом 16 нейронов")  
3. **Отделение логики поиска** от обучения модели  


---

## 🛠️ Реализация ConfigSampler

### Концепция
ConfigSampler - прослойка между:
1. Конфигами DEGANN (с метаданными tunable-параметров)
2. Алгоритмами поиска (Optuna и др.)

Решает:
- Автоматическое обнаружение параметров для оптимизации
- Преобразование "плоских" параметров поиска в сложные конфиги

### Основная реализация

In [ ]:
from copy import deepcopy
from dataclasses import fields, is_dataclass
from typing import Any, Dict, List, Union, TypeVar, Generic
from degann.networks.topology.tuning_utils import TuningMetadata, FieldMetadata

T = TypeVar('T')

class ConfigSampler(Generic[T]):
    """
    Bridges between configuration space and search algorithms.
    
    Responsibilities:
    1. Discovers tunable parameters via field metadata
    2. Transforms search algorithm's output into valid configurations
    
    Usage:
    >>> sampler = ConfigSampler(config)
    >>> concrete_config = sampler.sample_config({"param1": value1})
    """
    
    def __init__(self, generic_config: T):
        # Глубокое копирование исходного конфига
        self.generic_config = deepcopy(generic_config)
        self.variable_fields = self._find_variable_fields(self.generic_config)

    def _find_variable_fields(self, config: Any, prefix: str = "") -> Dict[str, FieldMetadata]:
        """Рекурсивно собирает все варьируемые поля и их метаданные."""
        variables = {}
        
        if not is_dataclass(config) or not hasattr(config, "tuning_metadata"):
            return variables
        
        tuning_metadata: TuningMetadata = config.tuning_metadata
        
        for field in fields(config):
            field_name = f"{prefix}{field.name}"
            field_value = getattr(config, field.name)
            
            # Поле помечено как tunable?
            if field.metadata.get("tunable", False):
                meta = tuning_metadata.get(field.name, None)
                if meta:
                    variables[field_name] = meta
            
            # Рекурсия для вложенных конфигов
            if is_dataclass(field_value):
                nested_vars = self._find_variable_fields(field_value, f"{field_name}.")
                variables.update(nested_vars)
        
        return variables

    def sample_config(self, values: Dict[str, Any]) -> T:
        """Создаёт конкретный конфиг на основе выбранных значений."""
        config_copy = deepcopy(self.generic_config)
        return self._apply_values(config_copy, values)

    def _apply_values(self, config: Any, values: Dict[str, Any], prefix: str = "") -> Any:
        """Рекурсивно применяет значения к варьируемым полям."""
        if not is_dataclass(config):
            return config
            
        for field in fields(config):
            field_name = f"{prefix}{field.name}"
            field_value = getattr(config, field.name)
            
            if field_name in values:
                setattr(config, field.name, values[field_name])
            elif is_dataclass(field_value):
                nested_config = self._apply_values(field_value, values, f"{field_name}.")
                setattr(config, field.name, nested_config)
        
        return config

## Тест базовой функциональности

In [ ]:
# Минимальный тест-конфиг для проверки ядра функциональности
from dataclasses import dataclass, field, InitVar
from typing import Union, Any, Optional


@dataclass 
class TestConfig:
    layers: int = field(default="1", metadata={"tunable": True})
    activation: str = field(default="relu", metadata={"tunable": True})
    metadata: InitVar[dict | None] = None
    tuning_metadata: Optional[TuningMetadata] = field(default=None, init=False)

    def __init__(self):
        metadata={
            "layers": FieldMetadata(
                length_boundary=(3, 3)
            ),
            "activation": FieldMetadata(
                choices=["tanh"]
           )
        }
        self.tuning_metadata = TuningMetadata(type(self))
        self.tuning_metadata.set_metadata(metadata)

# Проверка основных сценариев
def test_sampler_basic():
    # 1. Инициализация
    sampler = ConfigSampler(TestConfig())
    
    # 2. Корректное применение значений
    assert sampler.variable_fields.__contains__("layers")
    assert sampler.variable_fields.__contains__("activation")

    config = sampler.sample_config({"layers": 3, "activation": "tanh"})

    assert config.layers == 3
    assert config.activation == "tanh"
    
test_sampler_basic()

## 🧪 Практическая проверка: интеграция с GAN

### 1. Подготовка тестовых данных и конфигураций

In [ ]:
# Глобальные переменные для отслеживания лучшей модели
best_score = float("inf")
best_model = None

# Фиксированные параметры для теста
GEN_LAYERS = 4
GEN_NEURONS = 16  # не более 64
DISC_LAYERS = 4
DISC_NEURONS = 32

def create_gan_compile_config(learning_rate=1e-4):
    """Создает фиксированную конфигурацию компиляции для GAN"""
    return GANCompileParams(
        generator_params=DenseNetCompileParams(
            rate=learning_rate,
            optimizer="Adam",
            loss_func="BinaryCrossentropy",
            metric_funcs=["mean_absolute_error"],
        ),
        discriminator_params=DenseNetCompileParams(
            rate=learning_rate,
            optimizer="Adam",
            loss_func="BinaryCrossentropy",
            metric_funcs=["binary_accuracy"],
        )
    )

### 2. Определение конфигурации GAN с tunable параметрами

In [ ]:
def create_tunable_gan_config():
    """Создает базовую конфигурацию GAN с метаданными для поиска"""
    # Конфиг генератора
    gen_config = DenseNetParams(
        input_size=1,
        block_size=[GEN_NEURONS] * GEN_LAYERS,
        output_size=1,
        activation_func="leaky_relu",
        metadata={
            "block_size": FieldMetadata(
                choices=[
                    [16, 16],       # 2 слоя
                    [32, 64, 32],   # 3 слоя
                    [16, 32, 32, 16] # 4 слоя
                ]
            ),
            "activation_func": FieldMetadata(
                choices=["relu", "leaky_relu", "sigmoid", "tanh"]
            )
        }
    )

    # Конфиг дискриминатора
    disc_config = DenseNetParams(
        input_size=2,
        block_size=[DISC_NEURONS] * DISC_LAYERS,
        output_size=1,
        activation_func="leaky_relu",
        metadata={
            "block_size": FieldMetadata(
                choices=[
                    [16, 16],
                    [32, 64, 32],
                    [16, 32, 32, 16]
                ]
            ),
            "activation_func": FieldMetadata(
                choices=["relu", "leaky_relu", "sigmoid"]
            )
        }
    )

    return GANTopologyParams(
        generator_params=gen_config,
        discriminator_params=disc_config
    )

### 3. Objective-функция для Optuna

In [ ]:
import optuna
from degann.networks.topology.base_topology_configs import BaseTopologyParams

def gan_objective(trial: optuna.Trial, generic_config: BaseTopologyParams) -> float:
    """Функция для оптимизации архитектуры GAN"""
    global best_score, best_model
    
    # 1. Инициализация семплера
    sampler = ConfigSampler(generic_config)
    
    # 2. Подбор параметров через Optuna
    values = {}
    for field_name, field_meta in sampler.variable_fields.items():
        if field_meta.value_range:
            min_val, max_val, step = field_meta.value_range
            if isinstance(min_val, int):
                values[field_name] = trial.suggest_int(field_name, min_val, max_val, step)
            else:
                values[field_name] = trial.suggest_float(field_name, min_val, max_val, step)
        elif field_meta.choices:
            values[field_name] = trial.suggest_categorical(field_name, field_meta.choices)
    
    # 3. Создание конкретной конфигурации
    concrete_config = sampler.sample_config(values)
    concrete_compile_config = create_gan_compile_config()
    
    # 4. Создание и обучение модели
    model = IModel(concrete_config)
    model.compile(concrete_compile_config)
    
    history = model.train(
        X_train, y_train,
        epochs=100,
        mini_batch_size=64,
        verbose=0
    )
    
    # 5. Оценка и сохранение лучшей модели
    val_loss = model.evaluate(X_test, y_test)[1]
    
    if val_loss < best_score:
        best_score = val_loss
        best_model = model
    
    return val_loss


### 4. Запуск оптимизации

In [ ]:
# Инициализация базового конфига
base_config = create_tunable_gan_config()

# Настройка и запуск исследования Optuna
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: gan_objective(trial, base_config),
    n_trials=100,
    show_progress_bar=True,
    callbacks=[smart_callback]
)

print(f"Лучшая достигнутая ошибка: {best_score:.4f}")


---

## 💡 Future Improvements

### 🔧 Проблема текущей реализации
```python
# Требуется явно перечислять все варианты
metadata = {
    "block_size": FieldMetadata(choices=[
        [16, 16], 
        [32, 64, 32],  # Невозможно выразить правила генерации
        [16, 32, 32, 16]
    ])
}
```


### 🛠️ Предлагаемое решение: Иерархические диапазоны

#### Архитектура:
```mermaid
graph LR
    A[ConfigSampler] --> B[SmartRange]
    B --> C[decompose]
    B --> D[compose]
    C --> E[BasicRange]
    C --> F[Другие SmartRange]
    D --> G[Конкретное значение]
```

#### Реализация:
```python
class SmartRange:
    """Базовый класс для сложных диапазонов"""
    @abstractmethod
    def decompose(self) -> Dict[str, Union[BasicRange, SmartRange]]:
        """Разложение на компоненты"""
        
    @abstractmethod 
    def compose(self, values: Dict[str, Any]) -> Any:
        """Сборка значения из компонентов"""

class LayerSizeRange(SmartRange):
    """Пример: Динамическое создание архитектуры сети"""
    def __init__(self, len_range: tuple, neuron_step: int):
        self.len_range = BasicRange(*len_range)
        self.neuron_step = neuron_step

    def decompose(self):
        return {
            "num_layers": self.len_range,
            "base_neurons": BasicRange(16, 64, self.neuron_step)
        }

    def compose(self, values):
        return [values["base_neurons"]] * values["num_layers"]

```

### ⚠️ Защита от рекурсии

```python
class RecursionGuard:
    def __init__(self):
        self._seen = set()

    def check(self, obj):
        obj_id = id(obj)
        if obj_id in self._seen:
            raise RecursionError("Обнаружена циклическая зависимость")
        self._seen.add(obj_id)

# Использование в ConfigSampler
guard = RecursionGuard()
for field in fields(config):
    guard.check(field.metadata["range"])  # Проверка перед обработкой
```

### 🔄 Пример работы

## Конфигурация с умным диапазоном
```python
gen_config = DenseNetParams(
    block_size=LayerSizeRange((2,5), 16),  # От 2 до 5 слоев по 16 нейронов
    ...
)
```

## Внутри ConfigSampler:
1. Вызов layer_range.decompose() → {"num_layers": (2,5), "base_neurons": (16,64,16)}
2. Подбор значений алгоритмом поиска
3. layer_range.compose({"num_layers":3, "base_neurons":32}) → [32, 32, 32]


---

### 🔧 **Функция `generate_sample` - Сердце трансформации параметров**

**Контекст:**  
ConfigSampler обнаруживает tunable-параметры, но нужен механизм для:  
1. Преобразования "плоских" значений от алгоритма поиска → в сложные конфиги  
2. Поддержки разных типов параметров (числа, категории, диапазоны)  

**Решение:**  
```python
def generate_sample(trial: optuna.Trial, sampler: ConfigSampler) -> Any:
    """Связующее звено между алгоритмом поиска и DEGANN-конфигами"""
    # 1. Собираем значения для всех tunable-полей через trial.suggest_*
    # 2. Применяем их к базовому конфигу через sampler.sample_config()
```

**Реализация:**

In [ ]:
import optuna
from degann.networks.topology.base_topology_configs import BaseTopologyParams

def generate_sample(trial: optuna.Trial, sampler: ConfigSampler) -> Any:
    """
    Генерирует конкретный конфиг на основе параметров из алгоритма поиска.
    
    Args:
        trial: Объект Optuna Trial для подбора параметров.
        sampler: Экземпляр ConfigSampler с информацией о tunable-полях.
        
    Returns:
        Конкретный конфиг с подобранными параметрами.
        
    Процесс:
        1. Для каждого tunable-поля в конфиге:
           - Если задан диапазон (value_range), подбирает целое/вещественное значение.
           - Если заданы варианты (choices), выбирает категориальное значение.
        2. Создает конфиг с подобранными значениями.
    """
    values = {}
    for field_name, field_meta in sampler.variable_fields.items():
        if field_meta.value_range:
            min_val, max_val, step = field_meta.value_range
            if isinstance(min_val, int):
                values[field_name] = trial.suggest_int(field_name, min_val, max_val, step)
            else:
                values[field_name] = trial.suggest_float(field_name, min_val, max_val, step)
        elif field_meta.choices:
            values[field_name] = trial.suggest_categorical(field_name, field_meta.choices)
    
    return sampler.sample_config(values)


**Ключевые особенности:**  
✅ **Универсальность**  
- Работает с любым алгоритмом поиска (Optuna, CMA-ES, etc)  
- Поддерживает все типы параметров через FieldMetadata  

✅ **Инкапсуляция**  
- Скрывает сложность преобразования параметров  
- Не требует модификации при добавлении новых полей  



---

### 🧩 **Класс `HyperparameterSearch` - Фабрика оптимизации**

**Проблема:**  
Ручное управление:  
- Конфигами топологии/компиляции  
- Данными обучения  
- Треккингом лучшей модели  

**Архитектура решения:**  
```mermaid
graph LR
    A[Optuna] --> B[HyperparameterSearch]
    B --> C[ConfigSampler]
    B --> D[IModel]
    B --> E[Треккинг лучшей модели]
```

**Ключевые методы:**  
| Метод                | Роль                                                                 |
|----------------------|----------------------------------------------------------------------|
| `generate_configs()` | Создает связанные конфиги топологии и компиляции                     |
| `train_model()`      | Обучает модель и вычисляет метрику                                   |
| `get_objective()`    | Генерирует функцию для Optuna (интеграционная точка)                 |


**Реализация:**



In [ ]:
from typing import Dict, Any, Callable
import optuna
from dataclasses import asdict

class HyperparameterSearch:
    """
    Универсальный менеджер поиска гиперпараметров для DEGANN.
    
    Инкапсулирует:
    - Пространство поиска (топология + компиляция)
    - Данные для обучения
    - Логику создания и оценки моделей
    - Треккинг лучшей модели
    
    Пример использования:
    >>> search = HyperparameterSearch(topology_config, compile_config, train_data, IModel)
    >>> best_model, best_score = run_hyperparameter_search(search)
    """
    
    def __init__(
        self,
        topology_config: Any,
        compile_config: Any,
        train_data: Dict[str, Any],
        model_factory: Callable,
        best_model_storage: Dict[str, Any] = None
    ):
        """
        Инициализация поиска.
        
        Args:
            topology_config: Базовый конфиг топологии с tunable-полями.
            compile_config: Базовый конфиг компиляции с tunable-полями.
            train_data: Словарь с данными {"X_train": ..., "y_train": ..., ...}.
            model_factory: Функция создания модели (например, IModel).
            best_model_storage: Опциональное хранилище для лучшей модели.
        """
        self.topology_sampler = ConfigSampler(topology_config)
        self.compile_sampler = ConfigSampler(compile_config)
        self.train_data = train_data
        self.model_factory = model_factory
        self.best_model_data = best_model_storage or {
            "best_model": None, 
            "best_score": float("inf"),
            "best_configs": None
        }

    def generate_configs(self, trial: optuna.Trial) -> Dict[str, Any]:
        """Генерирует связанные конфиги топологии и компиляции."""
        return {
            "topology": generate_sample(trial, self.topology_sampler),
            "compile": generate_sample(trial, self.compile_sampler)
        }

    def train_model(self, configs: Dict[str, Any]) -> float:
        """Обучает модель и возвращает метрику качества."""
        model = self.model_factory(configs["topology"])
        model.compile(configs["compile"])
        
        history = model.train(
            self.train_data["X_train"],
            self.train_data["y_train"],
            epochs=self.train_data.get("epochs", 100),
            mini_batch_size=self.train_data.get("batch_size", 64),
            verbose=self.train_data.get("verbose", 0)
        )
        
        val_loss = model.evaluate(
            self.train_data["X_test"],
            self.train_data["y_test"]
        )[1]
        
        if val_loss < self.best_model_data["best_score"]:
            self.best_model_data.update({
                "best_model": model,
                "best_score": val_loss,
                "best_configs": configs
            })
        
        return val_loss

    def get_objective(self) -> Callable[[optuna.Trial], float]:
        """Возвращает функцию для оптимизации в Optuna."""
        def objective(trial: optuna.Trial) -> float:
            configs = self.generate_configs(trial)
            return self.train_model(configs)
        return objective


**Преимущества подхода:**  
1. **Гибкость**  
   - Поддержка любых архитектур через `model_factory`  
   ```python
   model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
   ```  
2. **Расширяемость**  
   - Добавление новых параметров через FieldMetadata  
3. **Сквозное тестирование**  
   ```python
   # Тест проверяет весь пайплайн:
   search = HyperparameterSearch(...)
   study = optuna.create_study()
   study.optimize(search.get_objective(), n_trials=10)
   ```



---

### 🧪 **Тестирование концепции - GAN как пример**

**Проверяем:**  
1. Корректность интеграции всех компонентов  
2. Работоспособность на реальной архитектуре  



In [ ]:
# 1. Подготовка данных и конфигов
train_data = {
    "X_train": X_train, "y_train": y_train,
    "X_test": X_test, "y_test": y_test,
    "epochs": 50, "batch_size": 32
}

# Для генератора
gen_config = DenseNetParams(
    input_size=1,
    block_size=[3] * 3,
    output_size=1,
    activation_func="leaky_relu",
    # Добавляем metadata для tunable полей
    metadata={
        "block_size": FieldMetadata(
            choices=[
                [16, 16],       # 2 слоя
                [32, 64, 32],    # 3 слоя
                [16, 32, 32, 16] # 4 слоя
            ]
        ),
        "activation_func": FieldMetadata(
            choices=["relu", "leaky_relu", "sigmoid", "tanh"]
        )
    }
)

# Для дискриминатора (аналогично)
disc_config = DenseNetParams(
    input_size=2,
    block_size=[3] * 3,
    output_size=1,
    activation_func="leaky_relu",
    metadata={
        "block_size": FieldMetadata(
            choices=[
                [16, 16],       # 2 слоя
                [32, 64, 32],    # 3 слоя
                [16, 32, 32, 16] # 4 слоя
            ]
        ),
        "activation_func": FieldMetadata(choices=["relu", "leaky_relu", "sigmoid"])
    }
)

base_topology_config = GANTopologyParams(
    generator_params=gen_config,
    discriminator_params=disc_config,
)


learning_rate = 1e-4
# Компиляция
gen_compile_config = DenseNetCompileParams(
    rate=learning_rate,
    optimizer="Adam",
    loss_func="BinaryCrossentropy",
    metric_funcs=["mean_absolute_error"],
)
    
disc_compile_config = DenseNetCompileParams(
    rate=learning_rate,
    optimizer="Adam",
    loss_func="BinaryCrossentropy",
    metric_funcs=["binary_accuracy"],
)
  
base_compile_config = GANCompileParams(
    generator_params=gen_compile_config,
    discriminator_params=disc_compile_config,
)



**Запуск поиска по данным:**

In [ ]:
# 2. Инициализация поиска
search = HyperparameterSearch(
    topology_config=base_topology_config,
    compile_config=base_compile_config,
    train_data=train_data,
    model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
)



# 3. Запуск Optuna
study = optuna.create_study(direction="minimize")
study.optimize(search.get_objective(), n_trials=20)

# 4. Доступ к лучшей модели
best_model = search.best_model_data["best_model"]
print("Best score:", search.best_model_data["best_score"])


**Выводы по тесту:**  
✅ **Рабочий пайплайн**  
- Optuna → ConfigSampler → IModel работает без ошибок  
- Лучшая модель сохраняется автоматически  



---

**Зададим функцию обобщенную:**   
🧩 **Она будет принимать:**
1) HyperparameterSearch
2) политику поиска 

🧩 **Возвращать:**  
1) нейросеть
2) ошибку на валидационных данных

Этой функцией примерно покажем как выглядит готовый результат системы Optuna + ConfigSampler

In [ ]:
from typing import Dict, Any, Callable, Tuple
import optuna
from optuna.samplers import BaseSampler

def run_hyperparameter_search(
    search: HyperparameterSearch,
    search_policy: Any = None,  # Может быть Optuna-семплер или кастомный объект
    n_trials: int = 100,
    direction: str = "minimize"
) -> Tuple[Any, float]:
    """
    Запускает поиск гиперпараметров и возвращает лучшую модель и её ошибку.

    Аргументы:
        search: Экземпляр HyperparameterSearch.
        search_policy: Политика поиска (например, optuna.samplers.TPESampler()).
        n_trials: Количество trials.
        direction: Направление оптимизации ("minimize" или "maximize").

    Возвращает:
        Лучшая модель и её ошибка (например, val_loss).
    """
    # Инициализация Optuna
    study = optuna.create_study(
        direction=direction,
        sampler=search_policy  # Например, TPESampler() или RandomSampler()
    )
    
    # Запуск оптимизации
    study.optimize(search.get_objective(), n_trials=n_trials)

    # Возвращаем лучшую модель и её ошибку
    return (
        search.best_model_data["best_model"],
        search.best_model_data["best_score"]
    )

**Небольшой тест, который показывает как выглядит вызов:**

In [ ]:
from optuna.samplers import TPESampler

# Пример: Запуск поиска для GAN  
best_model, best_score = run_hyperparameter_search(  
    search,
    search_policy=TPESampler(),  
    n_trials=15 
)  
print(f"Лучшая ошибка: {best_score:.4f}")  

# Вывод параметров лучшей модели  
if best_model:  
    print("Конфигурация генератора:", best_model.generator_config)  
    print("Конфигурация дискриминатора:", best_model.discriminator_config)  


---

## 🎯 Итоги и направления развития  

### **Что реализовано:**  
✅ **Универсальный поиск гиперпараметров**  
   - `ConfigSampler` автоматически определяет `tunable`-поля и преобразует их в конфиги.  
   - `HyperparameterSearch` объединяет логику обучения, оценки и трекинга лучшей модели.  
   - Интеграция с `Optuna` через `generate_sample`.  

✅ **Гибкость архитектуры**  
   - Поддержка любых моделей DEGANN через `model_factory`.  
   - Возможность добавлять новые параметры через `FieldMetadata`.  

### **Что можно улучшить:**  
🔧 **Динамические диапазоны**  
   - Реализация `SmartRange` для генерации сложных структур (например, `block_size` с переменной длиной).  
   - Автоматическая декомпозиция в "плоские" параметры для алгоритмов поиска.  

🔧 **Расширение поддержки алгоритмов**  
   - Добавление других оптимизаторов (например, `CMA-ES` без жёсткой привязки к `Optuna`).  

🔧 **Упрощение описания топологий**  
   - Более удобный синтаксис для `tunable`-параметров (например, через декораторы).  